# Recitation Week 3

This week we will review pattern matching and inductive data structures, and get some practice using them.

## Pattern Matching

Pattern matching is a control structure we will use many times in this class, especially when dealing with inductive structures.
Let's see some of the ways we can use pattern matching.

In [8]:
// What is the output of this code block? (Try to figure it out without running the code block)
// How could we modify this code so it outputs "The Pair of Numbers num1 and num2" when myList has 2 elements?
val myList = 131 :: 3155 :: Nil // or List(131, 3155)
myList match {
    case h :: Nil => s"The Number $h"
    case h1 :: h2 :: Nil => s"The Pair of Numbers $h1 and $h2"
    case h :: _ => "Multiple Numbers, with head " + h //underscore symbol is kinda like "anything else" 
}

myList: List[Int] = List(131, 3155)
res7_1: String = "The Pair of Numbers 131 and 3155"

### More advanced pattern matching (extra)

Pattern matching also has other features beyond those shown above.
Some that might come in handy later in the course are shown below.
In particular, pattern guards and matching multiple things in the same case.
It is also possible to match on the type of a variable if the type of the value being matched on is unknown.

Matching multiple patterns in a single case is done by placing a single vertical bar (`|`) between each pattern.  If any of the patterns is matched then the branch is taken. See line 6 in the code below.

Pattern guards are used to make a case more specific then a pattern allows.  
To use a guard we add the keyword `if` after the pattern and then put a boolean condition. If the pattern matches and the condition evaluates to `true`, then the case is taken.  Otherwise, the case is skipped.
One example is on line 7 in the code below.  

In [10]:
val numThings = -10
numThings match {
    case 0 => "Nothing here"
    case 1 => "One thing here"
    case 2 => "A couple things here"
    case 3 | 4 => "A few things here" // match on multiple patterns
    case n if n < 0 => "Less than zero things here!?" // with a guard
    case _ => "Many things here" // a default case
}

numThings: Int = -10
res9_1: String = "Less than zero things here!?"

## Inductive data structures
Pattern matching is particularly useful for interacting with inductive data structures.
Consider the following example, a list of integers.  (This is similar to how `List`s are implemented in Scala.)

$$\begin{array}{ccccc}
\textbf{NumList} & \rightarrow & Empty &\ |\  & Cons(\textbf{Num}, \textbf{NumList}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [11]:
sealed trait NumList
//traits are basically abstract class, you cannot created an instance of it, but you can inherit from it
//sealed because restricted to that file, different files cannot extend NumList


case object Empty extends NumList
//Empty is a class that extends/inherits from NumList
//case classes have alot of nice builtin features so we use case instead of just class so we don't have to do it ourselves
//object becasue not parameters

case class Cons(hd: Int, tl: NumList) extends NumList
//case class here because we have parameters here

defined trait NumList
defined object Empty
defined class Cons

In [12]:
// example lists

// using the :: (cons operator) and Nil cases
val scalaList = 131 :: 3155 :: Nil
// this is equivalent to using the List constructor
scalaList == List(131, 3155)

// the same list using our NumList data structure instead of the built in one
val myNumList = Cons(131, Cons(3155, Empty))

scalaList: List[Int] = List(131, 3155)
res11_1: Boolean = true
myNumList: Cons = Cons(hd = 131, tl = Cons(hd = 3155, tl = Empty))

### Expr
Now we will implement our own grammer.  For this exercise, consider a simple language representing arithmetic expressions.
Given the grammer below, write the corresponding Scala code (assume $\textbf{Num}$ can be represented by a Scala `Int` as in $\textbf{NumList}$).

$$\begin{array}{ccccccccc}
\textbf{Expr} & \rightarrow & Const(\textbf{Num}) &\ |\  & Plus(\textbf{Expr}, \textbf{Expr}) &\ |\  & Minus(\textbf{Expr}, \textbf{Expr}) &\ |\  & Times(\textbf{Expr}, \textbf{Expr}) \\
\textbf{Num} & \rightarrow & 0 \ |\ 1\ |\ 2\ |\ 3\ |\ 4\ |\ \cdots \\
\end{array}$$

In [15]:
sealed trait Expr
case class Const(n: Int) extends Expr
case class Plus(p1: Expr, p2: Expr) extends Expr
case class Minus(m1: Expr, m2: Expr) extends Expr
case class Times(t1: Expr, t2: Expr) extends Expr
case class Divide(num: Expr, denom: Expr) extends Expr

defined trait Expr
defined class Const
defined class Plus
defined class Minus
defined class Times
defined class Divide

In [16]:
// expressions using our new grammer
val myExpr0 = Const(0)
val myExpr1 = Plus(Const(500), Const(131)) // 500 + 131
val myExpr2 = Times(myExpr1, Minus(Const(105), Const(100))) // (500+131)*(105-100)

myExpr0: Const = Const(n = 0)
myExpr1: Plus = Plus(p1 = Const(n = 500), p2 = Const(n = 131))
myExpr2: Times = Times(
  t1 = Plus(p1 = Const(n = 500), p2 = Const(n = 131)),
  t2 = Minus(m1 = Const(n = 105), m2 = Const(n = 100))
)

In [17]:
// Come up with your own example
val myExpr3 = Plus(Times(Const(9),Const(3)), myExpr1) //(9*3 + myExpr1 which is 500+131)
val myExpr4= Divide(Const(10), Const(2))

myExpr3: Plus = Plus(
  p1 = Times(t1 = Const(n = 9), t2 = Const(n = 3)),
  p2 = Plus(p1 = Const(n = 500), p2 = Const(n = 131))
)
myExpr4: Divide = Divide(num = Const(n = 10), denom = Const(n = 2))

Now that we have our implementation of the grammer, we can write functions using this implementation.
We will often want to pattern match on the different cases of our grammer when writing these functions, as in the example below.

In [5]:
// Get all the constants in an expression
def constVals(e: Expr): List[Int] = {
    e match {
        case Const(n) => n :: Nil // or List(n)
        case Plus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Minus(e1, e2) => constVals(e1) ++ constVals(e2)
        case Times(e1, e2) => constVals(e1) ++ constVals(e2)
    }
}

defined function constVals

In [7]:
constVals(myExpr0)
constVals(myExpr2)
constVals(myExpr3)

res6_0: List[Int] = List(0)
res6_1: List[Int] = List(500, 131, 105, 100)
res6_2: List[Int] = List(9, 3, 500, 131)

## Eval
Given an expression, can we compute the corresponding value?
Can we write a function to compute the value (evaluate the expression)?

In [14]:
def eval(e: Expr): Int = e match {
    case Const(n) => n
    case Plus(e1, e2) => {eval(e1)+eval(e2)}
    case Minus(e1,e2) => {eval(e1)-eval(e2)}
    case Times(e1,e2) => {eval(e1)*eval(e2)}
    case Divide(nu,de) => {eval(nu)/eval(de)}
}

defined function eval

In [12]:
// test cases
eval(Const(42)) == 42
eval(myExpr0) == 0 // 0
eval(myExpr1) == 631 // 500 + 131
eval(myExpr2) == 3155 // (500+131)*(105-100)
eval(myExpr3) //631+27

res11_0: Boolean = true
res11_1: Boolean = true
res11_2: Boolean = true
res11_3: Boolean = true
res11_4: Int = 658

### Bonus

How would you add division or identifiers/variables to the Expr trait?

We will look at this more in the future, but thinking through this is a good way to practice.
(Hint: start by expanding the grammar and adding another case for the Expr trait.)